# OpenAI Functions Agent

This notebook showcases using an agent that uses the OpenAI functions ability to respond to the prompts of the user using a Large Language Model

Install openai,google-search-results packages which are required as the langchain packages call them internally

## Dependencis

>pip install openai google-search-results

>pip install faiss-cpu

## Example 1: OpenAI Function Agent with Tools

In [1]:
from langchain import (
    LLMMathChain,
    OpenAI,
    SerpAPIWrapper,
    SQLDatabase,
    SQLDatabaseChain,
)
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

The agent is given ability to perform 3 search functionalities with the respective tools

SerpAPIWrapper:
>This initializes the SerpAPIWrapper for search functionality (search).

LLMMathChain Initialization:
>This component provides math-related functionality.

SQL Database Initialization:
>This component provides the agent to query in Custom Data Base.


In [2]:
# Initialize the OpenAI language model
#Replace <your_api_key> in openai_api_key="<your_api_key>" with your actual OpenAI key.
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613",openai_api_key="<your_api_key>")

# Initialize the SerpAPIWrapper for search functionality
#Replace <your_api_key> in openai_api_key="<your_api_key>" with your actual SerpAPI key.
search = SerpAPIWrapper(serpapi_api_key="<your_api_key>")

# Initialize the LLMMathChain
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

# Initialize the SQL database using the Chinook database file
# Replace the file location to the custom Data Base
db = SQLDatabase.from_uri("sqlite:///../../../../../notebooks/Chinook.db")

# Initialize the SQLDatabaseChain with the OpenAI language model and SQL database
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

# Define a list of tools offered by the agent
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful when you need to answer questions about current events. You should ask targeted questions."
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="Useful when you need to answer questions about math."
    ),
    Tool(
        name="FooBar-DB",
        func=db_chain.run,
        description="Useful when you need to answer questions about FooBar. Input should be in the form of a question containing full context."
    )
]


In [6]:
mrkl = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [7]:
mrkl.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
)



> Entering new  chain...

Invoking: `Search` with `{'query': 'Leo DiCaprio girlfriend'}`


Amidst his casual romance with Gigi, Leo allegedly entered a relationship with 19-year old model, Eden Polani, in February 2023.
Invoking: `Calculator` with `{'expression': '19^0.43'}`




> Entering new  chain...
19^0.43```text
19**0.43
```
...numexpr.evaluate("19**0.43")...

Answer: 3.547023357958959
> Finished chain.
Answer: 3.547023357958959Leo DiCaprio's girlfriend is reportedly Eden Polani. Her current age raised to the power of 0.43 is approximately 3.55.

> Finished chain.


"Leo DiCaprio's girlfriend is reportedly Eden Polani. Her current age raised to the power of 0.43 is approximately 3.55."

## Example 2: Question Answering over documents with OpenAI Function Agents
Below showcases how to use Vectorstore `retrievers` for question answering of documents.

Start by ingesting your files into a Vectorstore and wrapping them around a Tool. Doing this will allow the OpenAI Agent to call on your functions thus allowing you to interact with your document. 

In [ ]:
from pydantic import BaseModel, Field

from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader

class DocumentInput(BaseModel):
    question: str = Field()


tools = []
files = [
    {"id": "1", "path": "../../../alphabet_earnings.pdf"}, 
    {"id": "2", "path": "../../../tesla_earnings.pdf"}
]

for file in files:
    loader = PyPDFLoader(file["path"])
    pages = loader.load_and_split()
    documents = [page.page_content for page in pages]
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)
    embeddings = OpenAIEmbeddings()
    retriever = FAISS.from_documents(docs, embeddings).as_retriever()
    
    """
    Wrap retrievers in a Tool
    """
    tools.append(
        Tool(
            args_schema=DocumentInput,
            name=file["name"], 
            description=f"useful when you want to answer questions about {file['name']}",
            func=retriever
        )
    )


Next, create your agent and specify the OpenAI agent type `AgentType.OPENAI_FUNCTIONS`.

In [ ]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo-0613", 
    openai_api_key="<your_api_key>"
)

agent = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
)

agent({"input": "What was the revenue increase of Alphabet and Tesla in Q1?"})